In [3]:
import requests
import os
import json
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_SECRET')
bearer_token = os.environ.get('TWITTER_BEARER_TOKEN')

search_url = "https://api.twitter.com/2/tweets/search/recent"

query_params = {'query': 'biden documents'}

In [4]:
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [5]:
def get_tweets(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [6]:
tweets = []
search_url = "https://api.twitter.com/2/tweets/search/recent"
search_term = 'ethereum'
query_params = {'query': search_term}

files = get_tweets(search_url, query_params)
# while files['meta']['next_token']:
for i in range(10):
    for item in files['data']:
        tweets.append([item['id'], item['text']])
    query_params = {'query': search_term, 'pagination_token':files['meta']['next_token']}
    files = get_tweets(search_url, query_params)
    


200
200
200
200
200
200
200
200
200
200
200


In [7]:
for i, element in enumerate(tweets):
    sentiment = get_tweet_sentiment(element[1])
    tweets[i] = [element[0], element[1], sentiment]

In [8]:
tweets[0]

['1641931621480775681',
 'RT @_ConsenSys: Today, we’re excited to announce that ConsenSys zkEVM is now Linea: an innovative Layer 2 solution for Ethereum, alongside…',
 'positive']

In [9]:
bad = 0
good = 0
blah = 0
for i, element in enumerate(tweets):
    sent = element[2]
    if sent == "neutral":
        blah += 1
    elif sent == "positive":
        good += 1
    elif sent == "negative":
        bad += 1

print(f'Good: {good} /n Bad: {bad} /n Blah: {blah}')


Good: 62 /n Bad: 2 /n Blah: 36
